"""This is a challenge, using all the ~knowledge~ I gained learning more about word embeddings and tflearn in 
the imdb review video, I will attempt this independently on an ign video game review dataset.  
"""


 Using the existing labels is extra credit. The baseline is that you can just convert the labels so that there are only 2 emotions (positive or negative). Ideally you can use an RNN via TFLearn like the one in this example, but I'll accept other types of ML models as well.

In [19]:
import pandas as pd
import numpy as np

In [20]:
data = pd.read_csv('ign.csv')

data = data[['title','score_phrase','score']]

In [21]:

#Start with binary classification. 
good_games = data.loc[data['score'] > 6.0]
bad_games = data.loc[data['score'] < 4.0]

In [22]:
print("There are ",len(bad_games)," negatively reviewed games vs. ",
      len(good_games)," positive.")

There are  1079  negatively reviewed games vs.  13324  positive.


In [72]:
data

,title,score_phrase,score
0,LittleBigPlanet PS Vita,Amazing,9.0
1,LittleBigPlanet PS Vita -- Marvel Super Hero E...,Amazing,9.0
2,Splice: Tree of Life,Great,8.5
3,NHL 13,Great,8.5
4,NHL 13,Great,8.5
5,Total War Battles: Shogun,Good,7.0
6,Double Dragon: Neon,Awful,3.0
7,Guild Wars 2,Amazing,9.0
8,Double Dragon: Neon,Awful,3.0
9,Total War Battles: Shogun,Good,7.0


In [23]:
data.score_phrase.value_counts()

Great          4773
Good           4741
Okay           2945
Mediocre       1959
Amazing        1804
Bad            1269
Awful           664
Painful         340
Unbearable       72
Masterpiece      55
Disaster          3
Name: score_phrase, dtype: int64

A comment. There are far more good games than bad games, so at worse picking every game as positive vs. negative would lead to 13/16 results, so some further process should be done, but for now just working with binary for a proof of concept. 

In [68]:
titles_good = good_games['title'].values
y_good = np.ones(len(titles_good))

titles_bad =bad_games['title'].values
y_bad = 0*np.ones(len(titles_bad))


X = np.concatenate((titles_good,titles_bad))
y = np.concatenate((y_good,y_bad))




In [69]:
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(X,y)

In [70]:
np.shape(y_train)

(10802,)

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()
vect.fit(X_train)
X_train = vect.transform(X_train)

max_value = X_train.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
feature_names = np.array(vect.get_feature_names())

print("lowest tfidf: " , feature_names[sorted_by_tfidf[:20]])
print("highest tfidf: " , feature_names[sorted_by_tfidf[20:]])



lowest tfidf:  ['nicey' 'spicy' 'amiyumi' 'amp' 'genie' 'puffy' 'snow' 'edtion' 'sensen'
 'chikko' 'kaihou' 'chibikko' 'daishuugou' 'garbage' 'hey' 'why' 'our'
 'make' 'daffy' 'dodgers']
highest tfidf:  ['attractive' 'strong' '1861' ..., 'drake' 'tmnt' 'futurama']


Now for some good old fashioned word2vec for these titles.

In [42]:
from gensim.models import Word2Vec

In [43]:
# train model
model = Word2Vec(X, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
#print(model['sentence'])

Word2Vec(vocab=93, size=100, alpha=0.025)
['L', 'i', 't', 'l', 'e', 'B', 'g', 'P', 'a', 'n', ' ', 'S', 'V', '-', 'M', 'r', 'v', 'u', 'p', 'H', 'o', 'E', 'd', 'c', ':', 'T', 'f', 'N', '1', '3', 'W', 's', 'h', 'G', '2', 'k', 'm', 'j', 'A', 'U', 'q', 'I', 'J', "'", 'z', 'D', '(', ')', 'y', 'C', 'R', 'F', 'b', 'O', '6', 'K', 'w', 'Y', 'x', '.', '5', '4', 'Q', '/', 'X', 'Z', '0', '!', '&', '?', '7', '9', '[', ']', '+', '8', '°', '@', ',', '*', '$', '\\', '#', '"', '~', '♥', 'Я', 'ñ', 'ü', 'ö', 'é', '’', ';']


In [41]:
X_vect[0]

<1x6713 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [15]:
max_title = 0
for game in titles:
    title_len = len(game)
    if title_len > max_title:
        max_title = title_len
        print("# of words:", len(game.split()))

# of words: 3
# of words: 8
# of words: 11
# of words: 11
# of words: 11
# of words: 12
# of words: 13
# of words: 16
# of words: 16


Now using these titles, 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
titles.values


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = TfidfVectorizer()

vect.fit(titles.values)


eh = vect.transform(titles.values)

In [ ]:
vect.fit_transform(titles.values)

In [ ]:
tits  =vect.fit_transform(titles.values)

In [ ]:
vect.get_feature_names()

The longest one is 96 characters, and 16 words. 

In [ ]:
from tflearn.datasets import imdb #a preprocessed data set. Something you don't see in nature! 

# IMDB Dataset loading
train, test, _ = imdb.load_data(path='imdb.pkl', n_words=10000,
                                valid_portion=0.1)  #load as a pkl file, a byte oriented object that can easily be converted to lists and tuples later on, or in this code!
				#n words is how many total words we want to look at, and validation corresponds to the size of the validation data to be placed inside, or.1 (test = .1*train in this case.)


#split features and labels. 
trainX, trainY = train
testX, testY = test 


In [ ]:
trainX[0]

In [ ]:
trainX = pad_sequences(trainX, maxlen=100, value=0.)
testX = pad_sequences(testX, maxlen=100, value=0.)


In [ ]:
from tflearn.data_utils import to_categorical, pad_sequences #helper functions


In [ ]:
trainX[0]

In [ ]:
trainX[1]